In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import *

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles_np = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=100)

In [6]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

# backtest_settings = BacktestSettings(
#     array_size=1000,
#     gains_pct_filter=0,
#     total_trade_filter=40,
#     upside_filter=0,
# )

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05,]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type='limit',
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [7]:
strat_records_df = nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    long_short="long",
    nb_strat_evaluate=nb_strat_long_evaluate,
    nb_strat_get_current_ind_settings=nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str=nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings=nb_strat_get_total_ind_settings,
    nb_strat_ind_creator=nb_strat_bt_create_ind,
    static_os=static_os,
)
strat_records_df.sort_values(by="qf_score", ascending=False).head(5)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 1
Total combinations of settings to test: 1
Total candles: 100
Total candles to test: 100
nb_strategy.py - nb_get_current_ind_settings() - Created indicator settings
nb_simulate.py - nb_run_backtest() - Indicator settings index=0
nb_strategy.py - nb_strat_get_ind_set_str() - RSI Period= 14 RSI is below= 70.000
nb_simulate.py - nb_run_backtest() - Dynamic Order settings index=0
nb_simulate.py - nb_run_backtest() - Created Dynamic Order Settings
max_equity_risk_pct= 12.000
max_trades= 5
risk_account_pct_size= 3.000
risk_reward= 2.500
sl_based_on_add_pct= 0.050
sl_based_on_lookback= 20
sl_bcb_type= Low
sl_to_be_cb_type= Nothing
sl_to_be_when_pct= 0.0
trail_sl_bcb_type= Low
trail_sl_by_pct= 0.500
trail_sl_when_pct= 1.000
nb_simulate.py - nb_run_backtest() - Starting Bar=50
nb_simulate.py - nb_run_backtest() - account state order results pnl array 

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
0,0,0,4.0,0,4,-24.413,0.0,-0.95,129.335,-244.133,755.867
